<a href="https://colab.research.google.com/github/aydanali/ECON3916-Stats-and-ML/blob/main/Lab9/Lab9_StatsML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Load your dataset here (ensure lalonde.csv is uploaded to Colab or linked)
from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_csv('gdrive/My Drive/lalonde.csv')

Mounted at /content/gdrive


In [3]:
df.describe()

,Unnamed: 0,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,307.500000,0.301303,27.363192,10.268730,0.395765,0.117264,0.415309,0.630293,4557.546569,2184.938207,6792.834483
std,177.390811,0.459198,9.881187,2.628325,0.489413,0.321997,0.493177,0.483119,6477.964479,3295.679043,7470.730792
min,1.000000,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,154.250000,0.000000,20.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,238.283425
50%,307.500000,0.000000,25.000000,11.000000,0.000000,0.000000,0.000000,1.000000,1042.330000,601.548400,4759.018500
75%,460.750000,1.000000,32.000000,12.000000,1.000000,0.000000,1.000000,1.000000,7888.498250,3248.987500,10893.592500
max,614.000000,1.000000,55.000000,18.000000,1.000000,1.000000,1.000000,1.000000,35040.070000,25142.240000,60307.930000


In [6]:
# Naive Comparison
treated = df.loc[df['treat'] == 1, 're78']
control = df.loc[df['treat'] == 0, 're78']

naive_diff = treated.mean() - control.mean()
print(f"Naive Difference in Means: ${naive_diff:,.2f}")

Naive Difference in Means: $-635.03


In [7]:
# Define covariates
X = df[['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']]
y = df['treat']

# Fit Propensity Model
logit = LogisticRegression(solver='liblinear')
logit.fit(X, y)

# Generate Scores
df['pscore'] = logit.predict_proba(X)[:, 1]

df[['treat', 'pscore']].head()

,treat,pscore
0,1,0.458074
1,1,0.151980
2,1,0.750966
3,1,0.674020
4,1,0.670590


In [9]:
# Separate groups
treated = df[df.treat==1]
control = df[df.treat==0]

# Fit NN on Control scores
nbrs = NearestNeighbors(n_neighbors=1).fit(control[['pscore']])

# Find matches for Treated scores
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()]

# Construct Matched DataFrame
matched_df = pd.concat([treated, matched_control])

,Unnamed: 0,treat,age,educ,black,hispan,married,nodegree,re74,re75,re78,pscore
0,1,1,37,11,1,0,1,1,0.0,0.0,9930.0460,0.458074
1,2,1,22,9,0,1,0,1,0.0,0.0,3595.8940,0.151980
2,3,1,30,12,1,0,0,0,0.0,0.0,24909.4500,0.750966
3,4,1,27,11,1,0,0,1,0.0,0.0,7506.1460,0.674020
4,5,1,33,8,1,0,0,1,0.0,0.0,289.7899,0.670590


In [10]:
from scipy import stats

# T-test on raw data
diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"Raw Effect (Difference): ${diff:,.2f}")
print(f"P-value: {p_val:.4f}")


# Isolate the matched outcomes
matched_treated = matched_df[matched_df.treat==1]['re78']
matched_control = matched_df[matched_df.treat==0]['re78']

# Estimate the causal effect (T-test on matched data)
matched_diff = matched_treated.mean() - matched_control.mean()
t_stat, p_val = stats.ttest_ind(matched_treated, matched_control)


print(f"Recovered Effect (Matched Difference): ${matched_diff:,.2f}")
print(f"P-value: {p_val:.4f}")

Raw Effect (Difference): $-635.03
P-value: 0.3342
Recovered Effect (Matched Difference): $625.05
P-value: 0.3979
